# Rob Hoffman Candles Strategy Backtest

In [1]:
import yfinance as yf
#df = yf.download(tickers='BTC-USD', period='2y', interval='1h')
df = yf.download(tickers='EURUSD=X', period='59d', interval='15m')
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2022-01-17 00:00:00+00:00,1.141683,1.141683,1.140771,1.140901,1.140901,0
2022-01-17 00:15:00+00:00,1.140771,1.140901,1.140771,1.140901,1.140901,0
2022-01-17 00:30:00+00:00,1.140771,1.141422,1.140771,1.141032,1.141032,0
2022-01-17 00:45:00+00:00,1.140901,1.141422,1.140901,1.141162,1.141162,0
2022-01-17 01:00:00+00:00,1.141032,1.141422,1.140901,1.141162,1.141162,0
...,...,...,...,...,...,...
2022-04-06 13:15:00+01:00,1.092657,1.093374,1.092657,1.093016,1.093016,0
2022-04-06 13:30:00+01:00,1.093016,1.093972,1.093016,1.093613,1.093613,0
2022-04-06 13:45:00+01:00,1.093374,1.093972,1.093255,1.093613,1.093613,0


In [2]:
#df=df[df['Volume']!=0]
#df.isna().sum()
df.reset_index(drop=True, inplace=True)
#df.tail()

In [3]:
import pandas_ta as ta
df["EMA"] = ta.ema(df.Close, length=20)
df["ATR"] = ta.atr(df.High, df.Low, df.Close, length=14)

In [4]:
backrollingN = 20
df['slopeEMA'] = df['EMA'].diff(periods=1)
df['slopeEMA'] = df['slopeEMA'].rolling(window=backrollingN).mean()

In [5]:
df.head()

,Open,High,Low,Close,Adj Close,Volume,EMA,ATR,slopeEMA
0,1.141683,1.141683,1.140771,1.140901,1.140901,0,NaN,NaN,NaN
1,1.140771,1.140901,1.140771,1.140901,1.140901,0,NaN,NaN,NaN
2,1.140771,1.141422,1.140771,1.141032,1.141032,0,NaN,NaN,NaN
3,1.140901,1.141422,1.140901,1.141162,1.141162,0,NaN,NaN,NaN
4,1.141032,1.141422,1.140901,1.141162,1.141162,0,NaN,NaN,NaN


## Total signal

In [13]:
TotSignal = [0] * len(df)
slopelimit=2e-5
percentlimit = 0.45
for row in range(0, len(df)):
    if df.slopeEMA[row] < -slopelimit and (min(df.Open[row], df.Close[row])-df.Low[row])/(df.High[row]-df.Low[row])>percentlimit:
        TotSignal[row]=1
    if df.slopeEMA[row] > slopelimit and (df.High[row]-max(df.Open[row], df.Close[row]))/(df.High[row]-df.Low[row])>percentlimit:
        TotSignal[row]=2

df['TotSignal']=TotSignal

D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning:

invalid value encountered in double_scalars

D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning:

invalid value encountered in double_scalars



In [14]:
df.tail(5)
df.iloc[2088]

Open         1.123343
High         1.123595
Low          1.121453
Close        1.121705
Adj Close    1.121705
Volume       0.000000
EMA          1.124650
ATR          0.001328
slopeEMA    -0.000108
TotSignal    0.000000
pointpos          NaN
Name: 2088, dtype: float64

## Visualizing signals on chart

In [15]:
import numpy as np
def pointpos(x):
    if x['TotSignal']==1:
        return x['High']+1e-3
    elif x['TotSignal']==2:
        return x['Low']-1e-3
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

In [16]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

dfpl = df[1800:2200]

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                go.Scatter(x=dfpl.index, y=dfpl.EMA, line=dict(color='orange', width=1), name="EMA")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="Signal")
fig.show()

In [17]:
# EMA Trend Signal

In [18]:
emasignal = [0]*len(df)
backcandles = 6

for row in range(backcandles, len(df)):
    upt = 1
    dnt = 1
    for i in range(row-backcandles, row+1):
        if df.High[i]>=df.EMA[i]:
            dnt=0
        if df.Low[i]<=df.EMA[i]:
            upt=0
    if upt==1 and dnt==1:
        #print("!!!!! check trend loop !!!!")
        emasignal[row]=3
    elif upt==1:
        emasignal[row]=2
    elif dnt==1:
        emasignal[row]=1

df['EMASignal'] = emasignal

In [19]:
def HoffmanBreakSignal(l, backcandles):
    for r in range(l-backcandles-1, l):
        if df.EMASignal[l]==2 and df.TotSignal[r]==2 and df.Close[l]>=df.High[r]:
            return 2
        elif df.EMASignal[l]==1 and df.TotSignal[r]==1 and df.Close[l]<=df.Low[r]:
            return 1

HoffmanBreak = [0]*len(df)
for row in range(backcandles, len(df)): #careful backcandles used previous cell
    HoffmanBreak[row] = HoffmanBreakSignal(row, backcandles)
df['HoffmanBreakSignal'] = HoffmanBreak

In [20]:
import numpy as np
def pointposbreak(x):
    if x['HoffmanBreakSignal']==1:
        return x['High']+1e-3
    elif x['HoffmanBreakSignal']==2:
        return x['Low']-1e-3
    else:
        return np.nan

df['pointposbreak'] = df.apply(lambda row: pointposbreak(row), axis=1)

In [21]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

dfpl = df[250:400]

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                go.Scatter(x=dfpl.index, y=dfpl.EMA, line=dict(color='orange', width=1), name="EMA")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointposbreak'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="Signal")
fig.show()

In [22]:
df.iloc[2089]

Open                  1.120699
High                  1.121453
Low                   1.120072
Close                 1.120951
Adj Close             1.120951
Volume                0.000000
EMA                   1.124298
ATR                   0.001350
slopeEMA             -0.000116
TotSignal             1.000000
pointpos              1.122453
EMASignal             0.000000
HoffmanBreakSignal         NaN
pointposbreak              NaN
Name: 2089, dtype: float64

In [23]:
SLSignal = [0] * len(df)
SLbackcandles = 4
for row in range(SLbackcandles, len(df)):
    mi=1e10
    ma=-1e10
    if df.HoffmanBreakSignal[row]==1:
        for i in range(row-SLbackcandles, row+1):
            ma = max(ma,df.High[i])
        SLSignal[row]=ma
    if df.HoffmanBreakSignal[row]==2:
        for i in range(row-SLbackcandles, row+1):
            mi = min(mi,df.Low[i])
        SLSignal[row]=mi
        
df['SLSignal']=SLSignal

In [24]:
dfpl = df[:]
def SIGNAL():
    return dfpl.HoffmanBreakSignal

In [25]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    initsize = 0.2
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        slatr = 4*self.data.ATR[-1]
        TPSLRatio = 1.5

        if self.signal1==2 and len(self.trades)==0:   
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=10000, margin=1/50, commission=.00)
stat = bt.run()
stat
#bt.plot()

D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Start                                     0.0
End                                    4626.0
Duration                               4626.0
Exposure Time [%]                   73.589799
Equity Final [$]                 12564.926858
Equity Peak [$]                  13234.720194
Return [%]                          25.649269
Buy & Hold Return [%]               -4.144798
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -29.002759
Avg. Drawdown [%]                   -3.388032
Max. Drawdown Duration                 1690.0
Avg. Drawdown Duration             165.615385
# Trades                                 47.0
Win Rate [%]                        46.808511
Best Trade [%]                       0.955136
Worst Trade [%]                     -0.643301
Avg. Trade [%]                    

In [26]:
bt.plot()

Row(id='1455', ...)

In [27]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    initsize = 0.2
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        TPSLRatio = 1.5

        if self.signal1==2 and len(self.trades)==0:   
            sl1 = self.data.SLSignal[-1]
            tp1 = self.data.Close[-1]+(self.data.Close[-1] - sl1)*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.SLSignal[-1]
            tp1 = self.data.Close[-1]-(sl1 - self.data.Close[-1])*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=10000, margin=1/50, commission=.00)
stat = bt.run()
stat

D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Start                                     0.0
End                                    4626.0
Duration                               4626.0
Exposure Time [%]                   50.486276
Equity Final [$]                  9519.696452
Equity Peak [$]                  10595.914023
Return [%]                          -4.803035
Buy & Hold Return [%]               -4.144798
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -23.399535
Avg. Drawdown [%]                    -7.22558
Max. Drawdown Duration                 2514.0
Avg. Drawdown Duration             743.166667
# Trades                                 88.0
Win Rate [%]                        40.909091
Best Trade [%]                       0.662308
Worst Trade [%]                       -0.6821
Avg. Trade [%]                    

In [28]:
bt.plot()

Row(id='2240', ...)